# Building POP-Terphenyl Polymers using mbuild

Here we use `mbuild` to read in a SMILES string of a terphenyl monomer and build an n-residue polymer from the monomer.

In [1]:
import mbuild as mb
import numpy as np
import subprocess
from mbuild.lib.recipes.polymer import Polymer
import warnings

warnings.filterwarnings('ignore')

First we get the POP-terphenyl monomer from a smiles string.

In [2]:
comp = mb.load('C(=O)c1ccc(c2c(cc(OC)cc2)(c3ccc(cc3)[C@H](N)C))cc1', smiles = True, name="POP")

I output all the indexes of hydrogen atoms because we will uses these indexes for extending the polymer later on.

In [3]:
for i, atom in enumerate(comp):
    if atom.name == "H":
        print(i, atom)

25 <H pos=([-0.2863 -0.53   -0.1319]), 1 bonds, id: 140593077943024>
26 <H pos=([-0.3535 -0.356   0.0328]), 1 bonds, id: 140593077941008>
27 <H pos=([-0.3417 -0.1275  0.1218]), 1 bonds, id: 140593077941392>
28 <H pos=([0.0238 0.3237 0.1286]), 1 bonds, id: 140593078093520>
29 <H pos=([-0.2587  0.6644  0.2184]), 1 bonds, id: 140593078092992>
30 <H pos=([-0.3616  0.5219  0.2592]), 1 bonds, id: 140593078092224>
31 <H pos=([-0.3482  0.5794  0.0869]), 1 bonds, id: 140593077941344>
32 <H pos=([-0.4049  0.3384  0.0985]), 1 bonds, id: 140593077942832>
33 <H pos=([-0.4091  0.1083  0.0184]), 1 bonds, id: 140593077940960>
34 <H pos=([ 0.1529  0.2134 -0.0776]), 1 bonds, id: 140593077940576>
35 <H pos=([ 0.365   0.0909 -0.0926]), 1 bonds, id: 140593078092608>
36 <H pos=([ 0.2052 -0.2164  0.1649]), 1 bonds, id: 140593078093616>
37 <H pos=([-0.006  -0.093   0.1809]), 1 bonds, id: 140593078094960>
38 <H pos=([ 0.4333 -0.2302  0.1013]), 1 bonds, id: 140593078095392>
39 <H pos=([ 0.6328 -0.1197  0.0441])

`mbuild` comes with a nice tool to visualize Compounds built into jupyter-notebooks. Using the object from `py3Dmol` we can coloro the atoms to identify the indices needed to make substiutions when building the polymer.

In [4]:
view = comp.visualize(show_ports=True)
style = {
                "stick": {"radius": 0.2, "color": "grey"},
                "sphere": {"scale": 0.3, "color" : "black"},
    }
view.setStyle({'model': -1, 'serial':26},style)
view.setStyle({'model': -1, 'serial':41},style)

nterm_connect = 41
cterm_connect = 26

We also make the two capping compounds using SMILES strings:

In [5]:
cap_o = mb.load('CO', smiles = True)
cap_o.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [6]:
cap_n = mb.load('CC(C)(C)OC=O', smiles = True)
cap_n.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Here we use the `Polymer` object to build a hexamer from the molecules we built in the previous cells. `Polymer.add_monomer()` is used to add the monomers to the polymer object. `Polymer.add_end_groups()` adds the capping groups to the object with `"head"` and `"tail"` labels for the each end of the polymer. `replace = True` will replace the specified atoms with the next residue. `indices` is used to specify which atom will be replaced in each group. Finally, when we call `Polymer.build()`, the polymer is built with the specified `n` residues and the provided capping residues. `sequence` is used if multiple monomers are provided.

In [7]:
chain = Polymer()
chain.add_monomer(compound=comp,
                  indices=[cterm_connect - 1, nterm_connect - 1],
                  separation=.15,
                  replace=True,
                  # orientation = [[0,-1,0],[1,0,0]]
                 )
chain.add_end_groups(compound = cap_o,
                     index = -1,
                     separation=0.15,
                     label="head",
                     duplicate = False
                    )

chain.add_end_groups(compound = cap_n,
                     index = -1,
                     separation=0.15,
                     label="tail",
                     duplicate = False
                    )

chain.build(n=6, sequence='A')
chain.energy_minimize()

In [8]:
chain.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [9]:
for p in chain.particles():
    print(p)

<C pos=([-0.2174 -0.4623 -0.1597]), 3 bonds, id: 140593038656944>
<O pos=([-0.1375 -0.5013 -0.2439]), 1 bonds, id: 140593038657136>
<C pos=([-0.2209 -0.3201 -0.1201]), 3 bonds, id: 140593038657280>
<C pos=([-0.3086 -0.2773 -0.019 ]), 3 bonds, id: 140593038657424>
<C pos=([-0.3046 -0.1457  0.0274]), 3 bonds, id: 140593038657568>
<C pos=([-0.2108 -0.0551 -0.0247]), 3 bonds, id: 140593038657712>
<C pos=([-0.1984  0.0829  0.0305]), 3 bonds, id: 140593038657856>
<C pos=([-0.0719  0.147   0.0506]), 3 bonds, id: 140593038658000>
<C pos=([-0.0706  0.2821  0.091 ]), 3 bonds, id: 140593038658144>
<C pos=([-0.189   0.3522  0.1155]), 3 bonds, id: 140593038658288>
<O pos=([-0.1802  0.482   0.1549]), 2 bonds, id: 140593038658432>
<C pos=([-0.287   0.5655  0.1809]), 4 bonds, id: 140593038658576>
<C pos=([-0.3122  0.2866  0.1   ]), 3 bonds, id: 140593038658720>
<C pos=([-0.3165  0.1539  0.0582]), 3 bonds, id: 140593038658864>
<C pos=([0.0593 0.0749 0.0355]), 3 bonds, id: 140593038659008>
<C pos=([ 0.1

In [10]:
# no need for rotation of peptide bonds
for bond in chain.bonds():
    if bond[0].name == "N":
        if bond[1].n_direct_bonds == 3:
            print("Rotating bond:", bond, "by", np.pi/2)
            chain.rotate_dihedral(bond, np.pi/2)
            
    if bond[1].name == "N":
        if bond[0].n_direct_bonds == 3:
            print("Rotating bond:", bond, "by", np.pi/2)
            chain.rotate_dihedral(bond, np.pi/2)       

Rotating bond: (<N pos=([ 0.5421 -0.0747  0.0411]), 3 bonds, id: 140593038660016>, <C pos=([ 0.5572 -0.0576  0.1766]), 3 bonds, id: 140593038666688>) by 1.5707963267948966
Rotating bond: (<N pos=([0.8824 0.3365 0.8863]), 3 bonds, id: 140593078338272>, <C pos=([1.0151 0.3653 0.866 ]), 3 bonds, id: 140593038747888>) by 1.5707963267948966
Rotating bond: (<N pos=([1.8304 0.524  1.1323]), 3 bonds, id: 140593038749232>, <C pos=([1.8266 0.6049 1.2432]), 3 bonds, id: 140593038945312>) by 1.5707963267948966
Rotating bond: (<N pos=([2.1596 1.1961 1.793 ]), 3 bonds, id: 140593038946512>, <C pos=([2.292  1.1675 1.8154]), 3 bonds, id: 140593039142832>) by 1.5707963267948966
Rotating bond: (<N pos=([3.0264 1.1839 2.2863]), 3 bonds, id: 140593039143792>, <C pos=([3.042  1.3081 2.343 ]), 3 bonds, id: 140593039355248>) by 1.5707963267948966
Rotating bond: (<N pos=([3.5337 1.9342 2.6989]), 3 bonds, id: 140593039357120>, <C pos=([3.6066 1.8872 2.8042]), 3 bonds, id: 140593038584384>) by 1.570796326794896

In [11]:
chain.visualize(show_ports=True)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Next we add specific residue labels for the componenets of the `Polymer` object. Here we label the monomers as HEX and the capping moieties as CAP.

In [12]:
print(chain.labels)
for label in chain.labels["monomer"]:
    label.name = "HEX"
    print(label)
for label in chain.labels["Compound"]:
    label.name = "CAP"
    print(label)

OrderedDict([('monomer', [<Compound 44 particles, 46 bonds, non-periodic, id: 140593038583616>, <Compound 44 particles, 46 bonds, non-periodic, id: 140593038573968>, <Compound 44 particles, 46 bonds, non-periodic, id: 140593038669040>, <Compound 44 particles, 46 bonds, non-periodic, id: 140593038945408>, <Compound 44 particles, 46 bonds, non-periodic, id: 140593039140432>, <Compound 44 particles, 46 bonds, non-periodic, id: 140593039355344>]), ('monomer[0]', <Compound 44 particles, 46 bonds, non-periodic, id: 140593038583616>), ('monomer[1]', <Compound 44 particles, 46 bonds, non-periodic, id: 140593038573968>), ('monomer[2]', <Compound 44 particles, 46 bonds, non-periodic, id: 140593038669040>), ('monomer[3]', <Compound 44 particles, 46 bonds, non-periodic, id: 140593038945408>), ('monomer[4]', <Compound 44 particles, 46 bonds, non-periodic, id: 140593039140432>), ('monomer[5]', <Compound 44 particles, 46 bonds, non-periodic, id: 140593039355344>), ('Compound', [<Compound 5 particles,

We save these as a pdb file and provide the names of the residues to include in the file.

In [13]:
chain.save("pop_hexamer_mbuild.pdb", overwrite=True, residues=["HEX", "CAP"])

Using Gromacs I generate a new gro file with the correct residue labels. `mbuild` doesn't seem to label residues correctly when writing `.gro` files.

In [14]:
! gmx editconf -f pop_hexamer_mbuild.pdb -o pop_hexamer_mbuild.gro

                     :-) GROMACS - gmx editconf, 2022.4 (-:

Executable:   /usr/local/bin/gmx
Data prefix:  /usr/local
Working dir:  /home/tfobe/Research/terphenyl_simulations/examples/build_polymer/pop
Command line:
  gmx editconf -f pop_hexamer_mbuild.pdb -o pop_hexamer_mbuild.gro

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 285 atoms
Volume: 65.7967 nm^3, corresponds to roughly 29600 electrons
No velocities found

Back Off! I just backed up pop_hexamer_mbuild.gro to ./#pop_hexamer_mbuild.gro.4#

GROMACS reminds you: "Pain is inevitable. Suffering is optional." (Haruki Murakami)



Lastly, I use openBabel to generate a `.mol` file for use in the OpenFF parameter assignment workflow.

In [15]:
! obabel -ipdb pop_hexamer_mbuild.pdb -omol pop_hexamer_mbuild.mol -O pop_hexamer_mbuild.mol

*** Open Babel Error  in ReadMolecule
  ERROR: not a valid PDB file

1 molecule converted


# Building arbitrary n-mers of POP monomer

Now we put it all together to generate a tetramer, hexamer and octamer PMP terphenyl foldamer:

In [16]:
n_residues = [4,6,8,10]
residue_labels = ["TET", "HEX", "OCT", "DEC"]
file_names = ["pop_tetramer_mbuild", "pop_hexamer_mbuild", "pop_octamer_mbuild", "pop_decamer_mbuild"]
for i in range(len(n_residues)):
    # Generate polymer
    chain = Polymer()
    chain.add_monomer(compound=comp,
                      indices=[cterm_connect - 1, nterm_connect - 1],
                      separation=.15,
                      replace=True,
                      # orientation = [[0,-1,0],[1,0,0]]
                     )
    chain.add_end_groups(compound = cap_o,
                         index = -1,
                         separation=0.15,
                         label="head",
                         duplicate = False
                        )

    chain.add_end_groups(compound = cap_n,
                         index = -1,
                         separation=0.15,
                         label="tail",
                         duplicate = False
                        )

    chain.build(n=n_residues[i], sequence='A')
    chain.energy_minimize()

    # Rotate peptide bond
    #for bond in chain.bonds():
    #    if bond[0].name == "N":
    #        if bond[1].n_direct_bonds == 3:
    #            print("Rotating bond:", bond, "by", np.pi/2)
    #            chain.rotate_dihedral(bond, np.pi/2)

    #    if bond[1].name == "N":
    #        if bond[0].n_direct_bonds == 3:
    #            print("Rotating bond:", bond, "by", np.pi/2)
    #            chain.rotate_dihedral(bond, np.pi/2)  


    # Relabel chains
    for label in chain.labels["monomer"]:
        label.name = residue_labels[i]
        print(label)
    for label in chain.labels["Compound"]:
        label.name = "CAP"
        print(label)
            
            
    chain.save(file_names[i] + ".pdb", overwrite=True, residues=[residue_labels[i], "CAP"])
    
    subprocess.run(["gmx", "editconf", "-f", file_names[i] + ".pdb", "-o", file_names[i]+ ".gro"])
    subprocess.run(["obabel", "-ipdb", file_names[i] + ".pdb", "-omol", file_names[i] + ".mol",  "-O", file_names[i] + ".mol"])
    subprocess.run(["renumber_pdb_atoms", "-f", file_names[i] + ".pdb", "-o", file_names[i] + "_renum.pdb"])

<TET 44 particles, 46 bonds, non-periodic, id: 140592682095696>
<TET 44 particles, 46 bonds, non-periodic, id: 140592682097568>
<TET 44 particles, 46 bonds, non-periodic, id: 140592682380128>
<TET 44 particles, 46 bonds, non-periodic, id: 140592682527152>
<CAP 5 particles, 4 bonds, non-periodic, id: 140592681952944>
<CAP 16 particles, 15 bonds, non-periodic, id: 140592681959136>
Note that major changes are planned in future for editconf, to improve usability and utility.
Read 197 atoms
Volume: 17.6158 nm^3, corresponds to roughly 7900 electrons
No velocities found


                     :-) GROMACS - gmx editconf, 2022.4 (-:

Executable:   /usr/local/bin/gmx
Data prefix:  /usr/local
Working dir:  /home/tfobe/Research/terphenyl_simulations/examples/build_polymer/pop
Command line:
  gmx editconf -f pop_tetramer_mbuild.pdb -o pop_tetramer_mbuild.gro


Back Off! I just backed up pop_tetramer_mbuild.gro to ./#pop_tetramer_mbuild.gro.3#

GROMACS reminds you: "Nobody Never Learnt No-Nothing from No History" (Gogol Bordello)

*** Open Babel Error  in ReadMolecule
  ERROR: not a valid PDB file

1 molecule converted


<HEX 44 particles, 46 bonds, non-periodic, id: 140593762005008>
<HEX 44 particles, 46 bonds, non-periodic, id: 140593762003280>
<HEX 44 particles, 46 bonds, non-periodic, id: 140593760534192>
<HEX 44 particles, 46 bonds, non-periodic, id: 140593760626928>
<HEX 44 particles, 46 bonds, non-periodic, id: 140593760727872>
<HEX 44 particles, 46 bonds, non-periodic, id: 140593760874128>
<CAP 5 particles, 4 bonds, non-periodic, id: 140593760387136>
<CAP 16 particles, 15 bonds, non-periodic, id: 140593762306368>
Note that major changes are planned in future for editconf, to improve usability and utility.
Read 285 atoms
Volume: 40.5641 nm^3, corresponds to roughly 18200 electrons
No velocities found


                     :-) GROMACS - gmx editconf, 2022.4 (-:

Executable:   /usr/local/bin/gmx
Data prefix:  /usr/local
Working dir:  /home/tfobe/Research/terphenyl_simulations/examples/build_polymer/pop
Command line:
  gmx editconf -f pop_hexamer_mbuild.pdb -o pop_hexamer_mbuild.gro


Back Off! I just backed up pop_hexamer_mbuild.gro to ./#pop_hexamer_mbuild.gro.5#

GROMACS reminds you: "I can't go to a restaurant and order food because I keep looking at the fonts on the menu. Five minutes later I realize that it's also talking about food." (Donald Knuth)

*** Open Babel Error  in ReadMolecule
  ERROR: not a valid PDB file

1 molecule converted


<OCT 44 particles, 46 bonds, non-periodic, id: 140593759486880>
<OCT 44 particles, 46 bonds, non-periodic, id: 140593759499024>
<OCT 44 particles, 46 bonds, non-periodic, id: 140593759698416>
<OCT 44 particles, 46 bonds, non-periodic, id: 140593759844672>
<OCT 44 particles, 46 bonds, non-periodic, id: 140593760025808>
<OCT 44 particles, 46 bonds, non-periodic, id: 140593760184320>
<OCT 44 particles, 46 bonds, non-periodic, id: 140593758354208>
<OCT 44 particles, 46 bonds, non-periodic, id: 140593758468896>
<CAP 5 particles, 4 bonds, non-periodic, id: 140593759354496>
<CAP 16 particles, 15 bonds, non-periodic, id: 140593759491728>
Note that major changes are planned in future for editconf, to improve usability and utility.
Read 373 atoms
Volume: 63.1806 nm^3, corresponds to roughly 28400 electrons
No velocities found


                     :-) GROMACS - gmx editconf, 2022.4 (-:

Executable:   /usr/local/bin/gmx
Data prefix:  /usr/local
Working dir:  /home/tfobe/Research/terphenyl_simulations/examples/build_polymer/pop
Command line:
  gmx editconf -f pop_octamer_mbuild.pdb -o pop_octamer_mbuild.gro


Back Off! I just backed up pop_octamer_mbuild.gro to ./#pop_octamer_mbuild.gro.2#

GROMACS reminds you: "Put Me Inside SSC, Let's Test Superstring Theory, Oh Yoi Yoi Accelerate the Protons" (Gogol Bordello)

*** Open Babel Error  in ReadMolecule
  ERROR: not a valid PDB file

1 molecule converted


<DEC 44 particles, 46 bonds, non-periodic, id: 140593756625632>
<DEC 44 particles, 46 bonds, non-periodic, id: 140593756634560>
<DEC 44 particles, 46 bonds, non-periodic, id: 140593756846832>
<DEC 44 particles, 46 bonds, non-periodic, id: 140593756988912>
<DEC 44 particles, 46 bonds, non-periodic, id: 140593757159328>
<DEC 44 particles, 46 bonds, non-periodic, id: 140593755254848>
<DEC 44 particles, 46 bonds, non-periodic, id: 140593755455344>
<DEC 44 particles, 46 bonds, non-periodic, id: 140593757155344>
<DEC 44 particles, 46 bonds, non-periodic, id: 140593755580752>
<DEC 44 particles, 46 bonds, non-periodic, id: 140593755324176>
<CAP 5 particles, 4 bonds, non-periodic, id: 140593756621072>
<CAP 16 particles, 15 bonds, non-periodic, id: 140593756627264>


                     :-) GROMACS - gmx editconf, 2022.4 (-:

Executable:   /usr/local/bin/gmx
Data prefix:  /usr/local
Working dir:  /home/tfobe/Research/terphenyl_simulations/examples/build_polymer/pop
Command line:
  gmx editconf -f pop_decamer_mbuild.pdb -o pop_decamer_mbuild.gro


Back Off! I just backed up pop_decamer_mbuild.gro to ./#pop_decamer_mbuild.gro.2#

GROMACS reminds you: "There's Nothing We Can't Fix, 'coz We Can Do It in the Mix" (Indeep)



Note that major changes are planned in future for editconf, to improve usability and utility.
Read 461 atoms
Volume: 101.105 nm^3, corresponds to roughly 45400 electrons
No velocities found


*** Open Babel Error  in ReadMolecule
  ERROR: not a valid PDB file

1 molecule converted
